In [1]:
import os
import sys; sys.path.insert(0, os.path.abspath("../"))
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from dataset.dataset import SkinLesion_Dataset, SegExamples
from pipeline.preprocessing import SkinLesionPreprocessing

preproc = SkinLesionPreprocessing()
examples = SegExamples()

In [18]:
from pipeline.feature_extraction import FeaturesExtraction
froc = FeaturesExtraction(color_params={}, lbp_params={}, glcm_params={})

ImportError: /home/alex/anaconda3/envs/cad1/lib/python3.9/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /home/alex/anaconda3/envs/cad1/lib/python3.9/site-packages/scipy/fft/_pocketfft/pypocketfft.cpython-39-x86_64-linux-gnu.so)

In [2]:
def gabor_calculate(im):
    df = pd.DataFrame()
    img2 = im.reshape(-1) # add the pixel values of the org image as feature.
    df['org_img'] = img2

    ksize = 3 # Keep it small not to miss small features
    sigma = 1 # Keep it small not to miss small features
    lamda = 1*np.pi/4
    gamma = 1 # if it is 1 meaning that it is looking for circular features, 0 otherwise.
    phi = 0
    num = 1
    for d in range(1,4,1):
        gabor_label = "Gabor"+str(num)
        theta = 1 * np.pi/d
        kernel = cv2.getGaborKernel((ksize,ksize), sigma, theta, lamda, gamma, phi, ktype=cv2.CV_32F)
        fimg = cv2.filter2D(im, cv2.CV_8UC3, kernel)
        filtered_img = fimg.reshape(-1) # 1d. add filtered image as feature
        df[gabor_label] = filtered_img
        num += 1
    return df

In [16]:

features = pd.DataFrame()
for i in tqdm(range(examples.seg_examples_df.shape[0])):
        if examples[i]['problem'] != 'binary':
            continue
        # labels.append(examples[i]['label'])
        image = examples[i]['img']
        image = preproc.preprocess(image) # the img uploaded as numpy array.
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # by converting to gray we dont lose information in gabor filters
        dataframe = gabor_calculate(gray)

        df1=dataframe.mean().to_frame().T
        df2=dataframe.var().to_frame().T
        df3=dataframe.skew().to_frame().T
        df4=dataframe.kurtosis(axis=0).to_frame().T
        df5=dataframe.min().to_frame().T

        frames = [df1, df2, df3, df4, df5]
        result = pd.concat(frames, axis=1, join='inner')
        features = pd.concat([features, result])
        # if i == 1:
        #     break

100%|██████████| 68/68 [01:43<00:00,  1.52s/it]


In [14]:
dataframe.dtypes

org_img    uint8
Gabor1     uint8
Gabor2     uint8
Gabor3     uint8
dtype: object

In [12]:
list(result.iloc[0,:].values)

[193.18401419672605,
 254.52443683905548,
 254.52305157178037,
 78.78613465160075,
 380.6245328818285,
 11.295502795186758,
 11.251076852787495,
 69.63110334417222,
 -1.5895063941536167,
 -8.943216724428503,
 -8.900661950851276,
 -1.5719537606887406,
 2.104967757329042,
 92.39230586573974,
 91.44267613073616,
 2.3307716692084424,
 103.0,
 190.0,
 189.0,
 25.0]

In [17]:
features

,org_img,Gabor1,Gabor2,Gabor3,org_img,Gabor1,Gabor2,Gabor3,org_img,Gabor1,Gabor2,Gabor3,org_img,Gabor1,Gabor2,Gabor3,org_img,Gabor1,Gabor2,Gabor3
0,134.314310,229.084444,229.086946,54.777594,1537.089684,4125.636717,4125.125494,259.954987,-2.195676,-2.384671,-2.384544,-2.164329,3.387853,4.009561,4.008645,3.343884,9,15,18,0
0,193.184014,254.524437,254.523052,78.786135,380.624533,11.295503,11.251077,69.631103,-1.589506,-8.943217,-8.900662,-1.571954,2.104968,92.392306,91.442676,2.330772,103,190,189,25
0,143.568231,246.584956,246.584658,58.551234,312.465075,665.289402,665.298410,54.748011,-2.738119,-3.608036,-3.608132,-2.555538,7.885171,12.661789,12.663056,7.248718,46,86,86,14
0,109.975288,200.219579,200.223545,44.851025,501.281377,1637.320379,1635.899263,89.636193,-0.443727,-0.471928,-0.471936,-0.433942,-1.153317,-1.178725,-1.179926,-0.953941,54,97,101,0
0,135.563797,234.947282,234.945379,55.286877,486.639991,553.260730,553.301991,84.846056,0.582341,-1.182804,-1.181599,0.510812,0.612809,0.591758,0.586079,0.585009,73,136,136,15
0,154.139953,237.936376,237.934496,62.863664,1322.580932,1075.595150,1075.764324,224.097724,-0.333220,-1.797121,-1.797048,-0.349553,-0.460999,1.845212,1.845227,-0.426635,55,101,101,3
0,151.317865,238.692793,238.697188,61.712298,1068.767092,899.211794,899.084573,183.433382,-0.301629,-1.682773,-1.683325,-0.299046,-0.494338,1.430357,1.431796,-0.414935,69,126,126,0
0,159.278334,253.185483,253.185678,64.958554,126.436345,106.600349,106.597553,23.132065,-3.170793,-6.311432,-6.311905,-2.770919,13.240120,40.357428,40.362544,11.065689,85,157,156,32
0,171.678701,244.408756,244.408988,70.016009,1381.151577,869.794462,869.710300,233.505970,-1.010925,-2.967852,-2.967764,-1.006354,0.550848,7.894379,7.893746,0.550912,41,79,75,0
0,170.688805,250.978033,250.979750,69.613013,773.177135,224.532781,224.699179,133.788706,-0.477980,-5.034998,-5.034936,-0.502473,0.182353,28.532927,28.518535,0.248352,51,96,94,0
